In [1]:
"""
Created on Thu Feb  4 06:20:12 2016

@author: monkey
"""
# % pylab inline 
# % matplotlib inline

import pandas as pd
import xgboost as xgb
import csv
import numpy as np
import pdb
from copy import deepcopy
import random
# import matplotlib.pyplot as plt
from sklearn.metrics import log_loss

from sklearn.cross_validation import cross_val_score
from xgboost import XGBClassifier
# from bayes_opt import BayesianOptimization

from sklearn import cross_validation
from sklearn import ensemble
from sklearn import metrics
from sklearn.cross_validation import StratifiedKFold

In [2]:
print('Load data...')
train = pd.read_csv("train.csv")
target = train['target']

train_with_target = train.drop(['ID'], axis = 1)
train = train.drop(['ID', 'target'], axis = 1)
test = pd.read_csv("test.csv")
ids = test['ID'].values
test = test.drop(['ID'], axis = 1)

# add number of nans
train['number_NAN'] = train.isnull().sum(axis = 1)
test['number_NAN'] = test.isnull().sum(axis = 1)
train_with_target['number_NAN'] = train.isnull().sum(axis = 1)

train_len = train.shape[0]
test_len = test.shape[0]

for elt in train.columns:
    train[elt + '_na'] = pd.isnull(train[elt]).astype(int)
    test[elt + '_na'] = pd.isnull(test[elt]).astype(int)

train = train.fillna(-1)
test = test.fillna(-1)
train_with_target = train_with_target.fillna(-1)

Load data...


In [3]:
sum(train.columns == test.columns)

264

In [4]:
train.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v123_na,v124_na,v125_na,v126_na,v127_na,v128_na,v129_na,v130_na,v131_na,number_NAN_na
0,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,...,0,0,0,0,0,0,0,0,0,0
1,-1.000000,-1.000000,C,-1.000000,9.191265,-1.000000,-1.000000,2.301630,-1.000000,1.312910,...,1,0,0,1,1,0,0,1,1,0
2,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,...,0,0,0,0,0,0,0,0,0,0
3,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,...,0,0,0,0,0,0,0,0,0,0
4,-1.000000,-1.000000,C,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.050328,...,1,1,0,1,1,1,0,1,1,0


# filter all object features

In [5]:
object_features = []
all_features = []
for feat in train.columns:
    if feat != 'number_NAN':
        all_features.append(feat)
    if train[feat].dtype == 'object':
        object_features.append(feat)

In [6]:
object_features.extend(['v38', 'v62', 'v72', 'v129'])
print object_features

['v3', 'v22', 'v24', 'v30', 'v31', 'v47', 'v52', 'v56', 'v66', 'v71', 'v74', 'v75', 'v79', 'v91', 'v107', 'v110', 'v112', 'v113', 'v125', 'v38', 'v62', 'v72', 'v129']


In [7]:
all_features = all_features[:131]

In [8]:
len(all_features)

131

In [9]:
sum(train.isnull().sum())

0

In [10]:
sum(test.isnull().sum())

0

# add frequencies for object features

In [11]:
train.columns

Index([u'v1', u'v2', u'v3', u'v4', u'v5', u'v6', u'v7', u'v8', u'v9', u'v10',
       ...
       u'v123_na', u'v124_na', u'v125_na', u'v126_na', u'v127_na', u'v128_na',
       u'v129_na', u'v130_na', u'v131_na', u'number_NAN_na'],
      dtype='object', length=264)

In [12]:
max_categories = 150

In [13]:
# prepare data
train_dummies = pd.DataFrame()
test_dummies = pd.DataFrame()

In [14]:
prob = train_with_target.groupby(['target']).size()[1]/float(sum(train_with_target.groupby(['target']).size()))

In [15]:
prob

0.76119872989214576

# try TSNE!!!!!!!!!!!!!!!!!!

In [16]:
train.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v123_na,v124_na,v125_na,v126_na,v127_na,v128_na,v129_na,v130_na,v131_na,number_NAN_na
0,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,...,0,0,0,0,0,0,0,0,0,0
1,-1.000000,-1.000000,C,-1.000000,9.191265,-1.000000,-1.000000,2.301630,-1.000000,1.312910,...,1,0,0,1,1,0,0,1,1,0
2,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,...,0,0,0,0,0,0,0,0,0,0
3,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,...,0,0,0,0,0,0,0,0,0,0
4,-1.000000,-1.000000,C,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.050328,...,1,1,0,1,1,1,0,1,1,0


In [17]:
test.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v123_na,v124_na,v125_na,v126_na,v127_na,v128_na,v129_na,v130_na,v131_na,number_NAN_na
0,1.375465e+00,11.361141,C,4.200778,6.57700,2.081784,1.784386,0.011094,9.523810,1.312911,...,0,0,0,0,0,0,0,0,0,0
1,-1.000000e+00,-1.000000,C,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,1.291029,...,1,1,0,1,1,1,0,1,1,0
2,-4.903407e-07,8.201529,C,4.544371,6.55010,1.558442,2.467532,0.007164,7.142858,1.575492,...,0,0,0,0,0,0,0,0,0,0
3,2.661870e+00,3.041241,C,1.657216,9.77308,2.078337,1.430855,1.252157,7.959596,1.575493,...,0,0,0,0,0,0,0,0,0,0
4,1.252822e+00,11.283352,C,4.638388,8.52051,2.302484,3.510159,0.074263,7.612904,1.050328,...,0,0,0,0,0,0,0,0,0,0


In [18]:
sum(train.isnull().sum())

0

In [19]:
sum(test.isnull().sum())

0

In [20]:
for elt in all_features:
    vector = pd.concat([train[elt], test[elt]], axis = 0)
    print str(elt)

    if len(vector.unique()) <= max_categories:
        train_dummies = pd.concat([train_dummies, pd.get_dummies(train[elt], prefix = elt, \
                                                                 dummy_na = True)], axis = 1).astype('int8')
        test_dummies = pd.concat([test_dummies, pd.get_dummies(test[elt], prefix = elt, \
                                                               dummy_na = True)], axis = 1).astype('int8')
        
        agrg = pd.concat([train[elt], target], axis = 1)
        agrg = agrg.groupby(by = elt, axis = 0).agg(['sum','count']).target
        agrg['weight'] = agrg.apply(lambda x: .5 + .5 * x['sum']/x['count'] if\
                                    (x['sum'] > x['count'] - x['sum']) else .5 + .5 * \
                                    (x['sum'] - x['count'])/x['count'], axis = 1)
        agrg.reset_index(inplace = True)
        train[elt + '_weight'] = pd.merge(train, agrg, how = 'left', on = elt)['weight']
        test[elt + '_weight'] = pd.merge(test, agrg, how = 'left', on = elt)['weight']
        train[elt + '_weight'] = train[elt + '_weight'].fillna(prob)
        test[elt + '_weight'] = test[elt + '_weight'].fillna(prob)
        
        del train[elt], test[elt]
        # train[elt], tmp_indexer = pd.factorize(train[elt])
        # test[elt] = tmp_indexer.get_indexer(test[elt])
    else:
        typ = str(train[elt].dtype)[:3]
        if (typ == 'flo') or (typ == 'int'):
            
            train[elt] = np.log(2.0 + train[elt])
            test[elt] = np.log(2.0 + test[elt])
        else:
            if (typ == 'obj'):
                
                # del train[elt], test[elt]
                train[elt + '_binary'], tmp_indexer = pd.factorize(train[elt])
                test[elt + '_binary'] = tmp_indexer.get_indexer(test[elt])
                
                list2keep = vector.value_counts()[ : max_categories].index
                train[elt] = train[elt].apply(lambda x: x if x in list2keep else np.nan)
                test[elt] = test[elt].apply(lambda x: x if x in list2keep else np.nan)                
                train_dummies = pd.concat([train_dummies, pd.get_dummies(train[elt], prefix = elt, \
                                                                         dummy_na = True)], axis = 1).astype('int8')
                test_dummies = pd.concat([test_dummies, pd.get_dummies(test[elt], prefix = elt, \
                                                                       dummy_na = True)], axis = 1).astype('int8')
                
                agrg = pd.concat([train[elt], target], axis = 1)
                agrg = agrg.groupby(by = elt, axis = 0).agg(['sum','count']).target
                agrg['weight'] = agrg.apply(lambda x: .5 + .5 * x['sum']/x['count'] if\
                                            (x['sum'] > x['count'] - x['sum']) else .5 + .5 * \
                                            (x['sum'] - x['count'])/x['count'], axis = 1)
                agrg.reset_index(inplace = True)
                train[elt + '_weight'] = pd.merge(train, agrg, how = 'left', on = elt)['weight']
                test[elt + '_weight'] = pd.merge(test, agrg, how = 'left', on = elt)['weight']
                train[elt + '_weight'] = train[elt + '_weight'].fillna(prob)
                test[elt + '_weight'] = test[elt + '_weight'].fillna(prob)
                
                del train[elt], test[elt]
                
            else:
                print('error', typ)
                
    if sum(test.isnull().sum()) > 0:
        pdb.set_trace()
    if sum(train.isnull().sum()) > 0:
        pdb.set_trace()

v1
v2
v3
v4
v5
v6
v7
v8
v9
v10
v11
v12
v13
v14
v15
v16
v17
v18
v19
v20
v21
v22
v23
v24
v25
v26
v27
v28
v29
v30
v31
v32
v33
v34
v35
v36
v37
v38
v39
v40
v41
v42
v43
v44
v45
v46
v47
v48
v49
v50
v51
v52
v53
v54
v55
v56
v57
v58
v59
v60
v61
v62
v63
v64
v65
v66
v67
v68
v69
v70
v71
v72
v73
v74
v75
v76
v77
v78
v79
v80
v81
v82
v83
v84
v85
v86
v87
v88
v89
v90
v91
v92
v93
v94
v95
v96
v97
v98
v99
v100
v101
v102
v103
v104
v105
v106
v107
v108
v109
v110
v111
v112
v113
v114
v115
v116
v117
v118
v119
v120
v121
v122
v123
v124
v125
v126
v127
v128
v129
v130
v131


In [21]:
len(train)

114321

In [22]:
sum(test_dummies.isnull().sum())

0

In [23]:
for elt in train.columns:
    if (elt[-2:] == 'na') & (elt != 'v2_na'):
        # print str(elt)
        dist = metrics.pairwise_distances(train.v2_na.reshape(1, -1), train[elt].reshape(1, -1))
        if dist < 8:
            print "delete " + str(elt)
            del train[elt],test[elt]

delete v1_na
delete v4_na
delete v6_na
delete v7_na
delete v9_na
delete v11_na
delete v13_na
delete v15_na
delete v17_na
delete v18_na
delete v19_na
delete v20_na
delete v26_na
delete v27_na
delete v28_na
delete v29_na
delete v32_na
delete v33_na
delete v35_na
delete v37_na
delete v39_na
delete v41_na
delete v42_na
delete v43_na
delete v44_na
delete v45_na
delete v48_na
delete v49_na
delete v53_na
delete v55_na
delete v57_na
delete v58_na
delete v59_na
delete v60_na
delete v61_na
delete v64_na
delete v65_na
delete v67_na
delete v68_na
delete v73_na
delete v76_na
delete v77_na
delete v80_na
delete v83_na
delete v84_na
delete v86_na
delete v88_na
delete v90_na
delete v92_na
delete v93_na
delete v94_na
delete v95_na
delete v96_na
delete v97_na
delete v99_na
delete v100_na
delete v101_na
delete v103_na
delete v104_na
delete v106_na
delete v111_na
delete v116_na
delete v118_na
delete v120_na
delete v121_na
delete v122_na
delete v126_na
delete v127_na
delete v130_na


In [24]:
train = pd.concat([train, train_dummies, target], axis = 1)
test = pd.concat([test, test_dummies], axis = 1)

In [25]:
train.shape

(114321, 787)

In [26]:
for elt in list(set(train.columns) - set(test.columns)):
    del train[elt]
for elt in list(set(test.columns) - set(train.columns)):
    del test[elt]

In [27]:
train.shape

(114321, 766)

In [28]:
test.shape

(114393, 766)

In [29]:
sum(train.isnull().sum())

0

In [30]:
sum(test.isnull().sum())

0

In [31]:
train.head()

,v1,v2,v4,v5,v6,v7,v8,v9,v10,v11,...,v129_1.0,v129_2.0,v129_3.0,v129_4.0,v129_5.0,v129_6.0,v129_7.0,v129_8.0,v129_11.0,v129_nan
0,1.204694,2.372808,1.778510,2.294076,1.525899,1.644205,0.699597,2.484907,0.917602,2.914203,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,0.000000,0.000000,2.415134,0.000000,0.000000,1.458994,0.000000,1.197827,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,1.079727,1.989254,1.858010,1.991451,1.788352,1.779783,0.702922,2.685577,1.017353,2.818762,...,0,1,0,0,0,0,0,0,0,0
3,1.028696,2.332606,1.828723,2.612085,1.410426,1.383177,0.775624,2.394755,2.145074,2.909492,...,1,0,0,0,0,0,0,0,0,0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.115249,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [32]:
test.head()

,v1,v2,v4,v5,v6,v7,v8,v9,v10,v11,...,v129_1.0,v129_2.0,v129_3.0,v129_4.0,v129_5.0,v129_6.0,v129_7.0,v129_8.0,v129_11.0,v129_nan
0,1.216533,2.592351,1.824675,2.149084,1.406534,1.330884,0.698679,2.444415,1.197827,2.936611,...,0,0,0,0,0,0,0,0,0,0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.191200,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,0.693147,2.322538,1.878605,2.145943,1.269323,1.496836,0.696723,2.212973,1.274103,2.841582,...,0,0,0,0,0,0,0,0,0,0
3,1.539417,1.617652,1.296702,2.465816,1.405689,1.232809,1.179318,2.298536,1.274103,2.798765,...,0,0,0,0,0,0,0,0,0,0
4,1.179523,2.586511,1.892869,2.353327,1.459192,1.706593,0.729606,2.263106,1.115249,2.867330,...,0,0,0,0,0,0,0,0,0,0


In [33]:
train.columns[100:]

Index([u'v122', u'v123', u'v124', u'v126', u'v127', u'v128', u'v130', u'v131',
       u'number_NAN', u'v2_na',
       ...
       u'v129_1.0', u'v129_2.0', u'v129_3.0', u'v129_4.0', u'v129_5.0',
       u'v129_6.0', u'v129_7.0', u'v129_8.0', u'v129_11.0', u'v129_nan'],
      dtype='object', length=666)

In [34]:
n_folds = 5
skf = list(StratifiedKFold(target, n_folds))
skf

[(array([ 22575,  22576,  22580, ..., 114318, 114319, 114320]),
  array([    0,     1,     2, ..., 22961, 22962, 22964])),
 (array([     0,      1,      2, ..., 114318, 114319, 114320]),
  array([22575, 22576, 22580, ..., 45728, 45729, 45731])),
 (array([     0,      1,      2, ..., 114318, 114319, 114320]),
  array([45724, 45726, 45730, ..., 68871, 68875, 68882])),
 (array([     0,      1,      2, ..., 114318, 114319, 114320]),
  array([68503, 68504, 68505, ..., 91556, 91559, 91560])),
 (array([    0,     1,     2, ..., 91556, 91559, 91560]),
  array([ 91424,  91425,  91426, ..., 114318, 114319, 114320]))]

In [35]:
len(skf[0][0])

91456

In [36]:
len(skf[0][1])

22865

In [38]:
rnd = 57
xgboost_params_1 = { 
        "objective": "binary:logistic",
        "booster": "gbtree",
        "eval_metric": "logloss",
        "eta": 0.01, # 0.06, #0.01,
        "min_child_weight": 1,
        "subsample": 0.9,
        "colsample_bytree": 0.4,
        "max_depth": 12,
        'silent': 0,
        'gamma': 0.1,
        'verbose_eval': True,
        'seed': rnd
    }

xgboost_params_2 = { 
        "objective": "binary:logistic",
        "booster": "gbtree",
        "eval_metric": "logloss",
        "eta": 0.01, # 0.06, #0.01,
        "min_child_weight": 1,
        "subsample": 0.9,
        "colsample_bytree": 0.2,
        "max_depth": 10,
        'silent': 0,
        'gamma': 0.1,
        'verbose_eval': True,
        'seed': rnd
    }

xgboost_params_3 = { 
        "objective": "binary:logistic",
        "booster": "gbtree",
        "eval_metric": "logloss",
        "eta": 0.01, # 0.06, #0.01,
        "min_child_weight": 1,
        "subsample": 0.9,
        "colsample_bytree": 0.5,
        "max_depth": 12,
        'silent': 0,
        'gamma': 0.1,
        'verbose_eval': True
    }

xgboost_params_4 = { 
        "objective": "binary:logistic",
        "booster": "gbtree",
        "eval_metric": "logloss",
        "eta": 0.01, # 0.06, #0.01,
        "min_child_weight": 1,
        "subsample": 0.9,
        "colsample_bytree": 0.5,
        "max_depth": 14,
        'silent': 0,
        'gamma': 0.1,
        'verbose_eval': True
    }


clfs=[
    'xgboost-1',
    'xgboost-2',
    ensemble.RandomForestClassifier(bootstrap=False, class_weight='auto', criterion='entropy',
            max_depth = None, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf = 4, min_samples_split = 4,
            min_weight_fraction_leaf=0.0, n_estimators = 3000, n_jobs=-1,
            oob_score=False, random_state=rnd, verbose=0,
            warm_start=False),
    ensemble.ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth = None, max_features='sqrt', max_leaf_nodes=None,
           min_samples_leaf = 4, min_samples_split = 4,
           min_weight_fraction_leaf=1e-5, n_estimators = 3000, n_jobs=-1,
           oob_score=False, random_state=rnd, verbose=0, warm_start=False),
    ensemble.GradientBoostingClassifier(init = None, learning_rate = 0.1, loss = 'deviance',
              max_depth = 30, max_features = None, max_leaf_nodes = None,
              min_samples_leaf = 4, min_samples_split = 4,
              min_weight_fraction_leaf = 0.0, n_estimators = 100,
              random_state = rnd, subsample = 0.8, verbose = 0,
              warm_start = False)
]



# clfs = ['xgboost-1', 'xgboost-2', 'xgboost-3', 'xgboost-4']

'''
clfs = ['xgboost-1']

'''


'''

clfs = [
    ensemble.GradientBoostingClassifier(init = None, learning_rate = 0.1, loss = 'deviance',
              max_depth = 15, max_features = None, max_leaf_nodes = None,
              min_samples_leaf = 1, min_samples_split = 3,
              min_weight_fraction_leaf = 0.0, n_estimators = 100,
              random_state = rnd, subsample = 0.8, verbose = 0,
              warm_start = False)
]


clfs=[
    ensemble.ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth = None, max_features='sqrt', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=1e-5, n_estimators = 1000, n_jobs=-1,
           oob_score=False, random_state=rnd, verbose=0, warm_start=False)
]
'''

'''
    ensemble.RandomForestClassifier(bootstrap=False, class_weight='auto', criterion='entropy',
            max_depth = None, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators = 3000, n_jobs=-1,
            oob_score=False, random_state=rnd, verbose=0,
            warm_start=False),
    ensemble.ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth = None, max_features='sqrt', max_leaf_nodes=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=1e-5, n_estimators = 3000, n_jobs=-1,
           oob_score=False, random_state=rnd, verbose=0, warm_start=False)
    '''

"\n    ensemble.RandomForestClassifier(bootstrap=False, class_weight='auto', criterion='entropy',\n            max_depth = None, max_features='sqrt', max_leaf_nodes=None,\n            min_samples_leaf=1, min_samples_split=2,\n            min_weight_fraction_leaf=0.0, n_estimators = 3000, n_jobs=-1,\n            oob_score=False, random_state=rnd, verbose=0,\n            warm_start=False),\n    ensemble.ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',\n           max_depth = None, max_features='sqrt', max_leaf_nodes=None,\n           min_samples_leaf=1, min_samples_split=2,\n           min_weight_fraction_leaf=1e-5, n_estimators = 3000, n_jobs=-1,\n           oob_score=False, random_state=rnd, verbose=0, warm_start=False)\n    "

In [39]:
clfs

['xgboost-1',
 'xgboost-2',
 RandomForestClassifier(bootstrap=False, class_weight='auto',
             criterion='entropy', max_depth=None, max_features='sqrt',
             max_leaf_nodes=None, min_samples_leaf=4, min_samples_split=4,
             min_weight_fraction_leaf=0.0, n_estimators=3000, n_jobs=-1,
             oob_score=False, random_state=57, verbose=0, warm_start=False),
 ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf=4, min_samples_split=4,
            min_weight_fraction_leaf=1e-05, n_estimators=3000, n_jobs=-1,
            oob_score=False, random_state=57, verbose=0, warm_start=False),
 GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
               max_depth=30, max_features=None, max_leaf_nodes=None,
               min_samples_leaf=4, min_samples_split=4,
               min_weight_fraction_leaf=0.0, n_estimators=100,

In [40]:
final_submit = []

In [41]:
xgb_rounds = 1500

In [ ]:
enumerate(skf[0])

In [ ]:
for i, (train_idx, test_idx) in enumerate(skf):
    print i
    X = train.iloc[train_idx]
    Y = train.iloc[test_idx]
    Xtarget = target.iloc[train_idx]
    Ytarget = target.iloc[test_idx]
    
    indice = 0
    #run models
    
    preds = []
    predstest = []
    
    for model in clfs:
        print indice
        if model == 'xgboost-1':
            dtrain = xgb.DMatrix(X, label = Xtarget)
            dcv = xgb.DMatrix(Y, label = Ytarget)
            watchlist = [(dtrain,'train'), (dcv,'eval')]
            clf_1 = xgb.train(xgboost_params_1, dtrain, num_boost_round = xgb_rounds, evals = watchlist, early_stopping_rounds = 100)
            preds.append(clf_1.predict(dcv, ntree_limit = clf_1.best_iteration))
        elif model == 'xgboost-2':
            dtrain = xgb.DMatrix(X, label = Xtarget)
            dcv = xgb.DMatrix(Y, label = Ytarget)
            watchlist = [(dtrain,'train'), (dcv,'eval')]
            clf_2 = xgb.train(xgboost_params_2, dtrain, num_boost_round = xgb_rounds, evals = watchlist, early_stopping_rounds = 100)
            preds.append(clf_2.predict(dcv, ntree_limit = clf_2.best_iteration))
        elif model == 'xgboost-3':
            dtrain = xgb.DMatrix(X, label = Xtarget)
            dcv = xgb.DMatrix(Y, label = Ytarget)
            watchlist = [(dcv,'eval'), (dtrain,'train')]
            clf_3 = xgb.train(xgboost_params_3, dtrain, num_boost_round = xgb_rounds, evals = watchlist, early_stopping_rounds = 100)
            preds.append(clf_3.predict(dcv, ntree_limit = clf_3.best_iteration))
        elif model == 'xgboost-4':
            dtrain = xgb.DMatrix(X, label = Xtarget)
            dcv = xgb.DMatrix(Y, label = Ytarget)
            watchlist = [(dcv,'eval'), (dtrain,'train')]
            clf_4 = xgb.train(xgboost_params_4, dtrain, num_boost_round = xgb_rounds, evals = watchlist, early_stopping_rounds = 100)
            preds.append(clf_4.predict(dcv, ntree_limit = clf_4.best_iteration))
        else:
            # pdb.set_trace()
            model.fit(X, Xtarget)
            preds.append(model.predict_proba(Y)[:,1])
            
        print('model ',indice,': loss=',metrics.log_loss(Ytarget, preds[indice]))
        
        if model == 'xgboost-1':
            dtest = xgb.DMatrix(test)
            predstest.append(clf_1.predict(dtest, ntree_limit = clf_1.best_iteration))
        elif model == 'xgboost-2':
            dtest = xgb.DMatrix(test)
            predstest.append(clf_2.predict(dtest, ntree_limit = clf_2.best_iteration))
        elif model == 'xgboost-3':
            dtest = xgb.DMatrix(test)
            predstest.append(clf_3.predict(dtest, ntree_limit = clf_3.best_iteration))
        elif model == 'xgboost-4':
            dtest = xgb.DMatrix(test)
            predstest.append(clf_4.predict(dtest, ntree_limit = clf_4.best_iteration))
        else:
            predstest.append(model.predict_proba(test)[:,1])
        
        indice += 1
        
    # find best weights
    step = 0.1 * (1./len(preds))
    print("step: ", step)
    poidsref = np.zeros(len(preds))
    poids = np.zeros(len(preds))
    poidsreftemp = np.zeros(len(preds))
    poidsref = poidsref + 1./len(preds)
        
    bestpoids = poidsref.copy()
    blend_cv = np.zeros(len(preds[0]))
        
    for k in range(0, len(preds), 1):
        blend_cv = blend_cv + bestpoids[k] * preds[k]
        bestscore = metrics.log_loss(Ytarget.values, blend_cv)
            
    getting_better_score = True
    while getting_better_score:
        getting_better_score = False
        for i in range(0, len(preds), 1):
            poids = poidsref
            if poids[i] - step>-step:
                # decrease weight in position i
                poids[i] -= step
                for j in range(0, len(preds), 1):
                    if j != i:
                        if poids[j] + step <= 1:
                            # try an increase in position j
                            poids[j] += step
                            # score new weights
                            blend_cv = np.zeros(len(preds[0]))
                            for k in range(0, len(preds), 1):
                                blend_cv = blend_cv + poids[k] * preds[k]
                            actualscore = metrics.log_loss(Ytarget.values, blend_cv)
                            # if better, keep it
                            if actualscore < bestscore:
                                bestscore = actualscore
                                bestpoids = poids.copy()
                                getting_better_score = True
                            poids[j] -= step
                poids[i] += step
        poidsref = bestpoids.copy()
            
    print("weights: ", bestpoids)
    print("optimal blend loss: ", bestscore)
    
    blend_to_submit = np.zeros(len(predstest[0]))
    for i in range(0, len(preds), 1):
        blend_to_submit = blend_to_submit + bestpoids[i] * predstest[i]
        
    final_submit.append(blend_to_submit)

0
0

Will train until eval error hasn't decreased in 100 rounds.
[0]	train-logloss:0.689227	eval-logloss:0.689665
[1]	train-logloss:0.684972	eval-logloss:0.686033
[2]	train-logloss:0.680827	eval-logloss:0.682395
[3]	train-logloss:0.676602	eval-logloss:0.678798
[4]	train-logloss:0.673005	eval-logloss:0.675600
[5]	train-logloss:0.669123	eval-logloss:0.672325
[6]	train-logloss:0.665736	eval-logloss:0.669282
[7]	train-logloss:0.662125	eval-logloss:0.666251
[8]	train-logloss:0.658202	eval-logloss:0.662992
[9]	train-logloss:0.654392	eval-logloss:0.659756
[10]	train-logloss:0.651287	eval-logloss:0.657056
[11]	train-logloss:0.647581	eval-logloss:0.653981
[12]	train-logloss:0.644678	eval-logloss:0.651400
[13]	train-logloss:0.641221	eval-logloss:0.648473
[14]	train-logloss:0.638195	eval-logloss:0.645905
[15]	train-logloss:0.635273	eval-logloss:0.643279
[16]	train-logloss:0.631908	eval-logloss:0.640487
[17]	train-logloss:0.628738	eval-logloss:0.637909
[18]	train-logloss:0.625507	eval-logloss:0.635221


In [ ]:
submit = np.mean(final_submit, axis = 0)

In [ ]:
submit

In [ ]:
# submit
submission = pd.read_csv('sample_submission.csv')
submission.PredictedProb = submit
submission.to_csv('blend.csv', index = False)